In [91]:
#### specify path of Julia Code for 2D Models ####
### By default the working directory ./2D_Models ###
path = string( pwd(), "/2D_Models/")

"/home/jakob/Dokumente/Repositories/Modeling_Framework/2D_Models/"

In [92]:
##### include 2D Modelling Framework ####
include(string(path, "./2D_model_library.jl"));


########## OPERATIONAL ##########
##### how to chose the actual velocity? #####

#op_model = "Constant"
#op_model = "Collision_Free_Speed"
#op_model = "RVO"
#op_model = "SocialForce"
op_model = "AV_Model"
#op_model = "AV_IN_Model"
#op_model = "PowerLaw"
#op_model = "IN_Costfunction"
#op_model = "IN_Model"
#op_model = "SocialForce"
#op_model = "SocialForce_Elliptical_A"
#op_model = "SocialForce_Elliptical_B"
#op_model = "SocialForce_CollisionPrediction"


include_op_model(op_model, path)

########## TACTICAL ##########
#### how to chose the preferred velocity? ###

tact_model = "Constant"
#tact_model = "IN_Model"

include_tact_model(tact_model, path)


########## STRATEGIC ##########
### how to chose the desired velocity ###

strat_model = "Constant"
#strat_model = "Towards_Goal"
#strat_model = "Two_Goals"

include_strat_model(strat_model, path)


########## OPTIMIZATION SCHEME ##########
### If optimization of a cost-function is included, how to solve? ###

#optimization_scheme = "Regular_Sampling"
optimization_scheme = "Random_Sampling"

include_optimization_scheme(optimization_scheme, path)


##### which update scheme? #####

update = "Parallel_Update"
#update = "Step_Update"

include_update(update, path)


##### which order? i.e. Acceleration or Velocity based? ####
#order = "first"
order = "second"

include_order(order, path)


##### PERCEPTIONAL MODEL ####
perception_model = "None"
#perception_model = "Only_in_Front"

include_perception_model(perception_model, path)

Typical parameters would be:
[2.0, 1.7, 3.0, 0.0, 0.3, 0.0, 0.0, 0.1, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0]
v_max, v_des, T, T2, l, step_time, τ_A, τ_R, α, β, ζ_h, ζ_v, r, λ


In [93]:
#### Special Scenario? ####
x = "Obstacle_Intruder"
#x = "Diminish_ANT_LF"

load_scenario(x, path)

In [94]:
ρ = 2

max(1/√ρ - 0.25, 0.3)

0.45710678118654746

In [120]:
#### parameters of the agents ####
p = [1.0, 0.0, 3.0, 0.0, 0.3, 0.2, 0.0, 0.1, 0.02, 1.0, 0.0, 0.0, 0.5, 0.0]
# ModelParameter: v_max, v_des, T, T2, l, l_min, τ_A, τ_R, α, β, ζ_h, ζ_v, r, ϕ
p_desc = "v_max, v_des, T, T2, l, l_min, τ_A, τ_R, α, β, ζ_h, ζ_v, r, ϕ"

### of the system ###
ρ = 1.5
system_size = (8.0, 4.0) #important for periodic boundaries
N = Int(round(ρ*system_size[1]*system_size[2]))

### of the simulation ###
sim_p = [10.0, 10.0 + 8.1/0.3, 0.05, 0.01, 3.0] #t_relax, t_max, dt_save, dt, r
sim_p_desc = "t_relax, t_max, dt_save, dt, r"

### initialize geometry ###
geometrie = create_geometry_single_obstacle((37.5,31.5), 0.3)

### initialize crowd ###
menge = create_crowd(N, geometrie)

Init_Hom_Parameters!(p, menge)
Init_Hom_Vels!(menge, 0.1)
Init_E_des!(menge, (1.0, 0.0))

#initialize_random_positions(system_size, menge, geometrie)
#CrowdOneSide!(menge, 0.5, 0.0, 4.0)
Init_Homogeneous_Random_InitialPositions_INTRUDER(menge, geometrie, system_size, 0.25)

Init_Random_Headings!(menge)

vel, l_obs = 0.3, 0.7
Init_Nth_Agent_as_Moving_Obstacle!(menge, system_size, l_obs, vel);

1.5033207337780499


In [121]:
positions, headings = Simulate!(menge, geometrie, sim_p[1], sim_p[2], sim_p[3], sim_p[4], sim_p[5], system_size);

In [ ]:
using GR
GR.inline("mov")

j = 1

dt_save = sim_p[3]
geometry_x, geometry_y = rewrite_geometry(geometrie)
boundaries_x, boundaries_y = boundaries_rectangle_positions(system_size)

while j*sim_p[3] < sim_p[2]-sim_p[1]
    
        GR.clearws()

        GR.setwindow(-1, 11, -1, 11)
        GR.setviewport(0.0, 1, 0.0, 1)
    

        plot_boundaries(boundaries_x, boundaries_y, 0.2, 999)        
        plot_geometry(geometry_x, geometry_y, 3.7, 20)

        plot_agents(positions[j, 1:N-1], headings[j, 1:N-1], 2.3, 100, 0.7, 999, 0.1)
    
        #plot moving obstacle
        plot_agents(positions[j, N:end], headings[j, N:end], 3.7, 97, 0.0, 999, 0.1)

        model = op_model
        timer = string("time ", round(j*dt_save, digits = 1), "s")
        GR.text(0.5, 0.95, timer)
        GR.text(0.5, 0.9, model)
        
        GR.updatews()
    
        j = j+1
end

In [ ]:
GR.show()

In [ ]:
Path = "/home/jakob/Dokumente/Sim_Data/"
dir = "Intruder_NO_ANT_4"
header = Header(p, p_desc, sim_p, sim_p_desc, op_model, tact_model, order, update, N, system_size);

Save_Data!(Path, dir, header, positions)

In [ ]:
samples = 20

for i in 1:samples
    
    Random.seed!()
    
    #if mod(i, 10) == 0
        println(i)
    #end
    
    Init_Hom_Vels!(menge, 0.0)
    Init_Rand_Rect!(a0, a, b0, b, menge, geometrie)
    initialize_random_headings(menge)

    vel, l_obs = 0.3, 0.7

    Init_Nth_Agent_as_Moving_Obstacle!(menge, system_size, l_obs, vel);
    
    positions, headings, ttcs = Simulate!(menge, geometrie, sim_p[1], sim_p[2], sim_p[3], sim_p[4], sim_p[5], system_size);
    Save_Data!(Path, dir, header, positions)
    
end

In [46]:
# if the system starts at (0, 0) i.e. x_{min}
function Init_Homogeneous_Random_InitialPositions(menge::crowd, geometrie::geometry, system_size::NTuple{2, Float64},
        ε::Float64)

    ρ = ρ_global(menge, system_size, geometrie)
    d_des = max(1/√ρ - ε, menge.agent[1].l)
    
    println(ρ)
    
    if ρ > 7.5
        println("The density is too high!")
        false
    end

    for (i, a) in enumerate(menge.agent)

       a.pos = rand()*system_size[1] , rand()*system_size[2]

        while Too_Close(i, menge, geometrie, system_size, d_des)
            a.pos = rand()*system_size[1] , rand()*system_size[2]
        end

    end
end

Init_Homogeneous_Random_InitialPositions (generic function with 1 method)

In [47]:
# if the system starts at (0, 0) i.e. x_{min}
function Init_Homogeneous_Random_InitialPositions_INTRUDER(menge::crowd, geometrie::geometry, system_size::NTuple{2, Float64},
        ε::Float64)

    ρ = ρ_global(menge, system_size, geometrie)
    d_des = max(1/√ρ - ε, menge.agent[1].l)
    p_intruder = (system_size[1], system_size[2]/2)
    
    println(ρ)
    
    if ρ > 7.5
        println("The density is too high!")
        false
    end

    for (i, a) in enumerate(menge.agent)

       a.pos = rand()*system_size[1] , rand()*system_size[2]

        while Too_Close(i, menge, geometrie, system_size, d_des) || d(a.pos, p_intruder, system_size) < d_des
            a.pos = rand()*system_size[1] , rand()*system_size[2]
        end

    end
end

Init_Homogeneous_Random_InitialPositions_INTRUDER (generic function with 1 method)

In [7]:
max(1/√2 - 0.2, 0.3)

0.5071067811865475

In [ ]:
for a in menge.agent
    if Min_R(a, menge, geometrie, system_size) < max(1/√ρ - 0.1, a.l)
        println("JA")
    end
end

In [12]:
function Too_Close(current_agent::Int, menge::crowd, geometrie::geometry, system_size, d_des)

    for i in 1:current_agent-1

        if d(menge.agent[current_agent], menge.agent[i], system_size) < d_des
            return true
        end
    end

    for x in geometrie.element

        if d(menge.agent[current_agent], x, system_size) < d_des
            return true
        end
    end

    false
end

Too_Close (generic function with 2 methods)